In [9]:
'''
Author  : Jiayao Yu, User Interfaces Group, Aalto University
Init    : August, 2017
Project : ELEC-D7010 Engineering for Humans course materials
Topic   : Multiple Resource Theory
===========================================================================
Usage explaination
------
where's reference from?
Multiple Resource Theory is to predict the relative differences in task
interference between different task configurations.

parameters shorthand explainations
------
      Task Analysis Shell: PC = Perception & Cognition, R = Responding,
                          V (under Modalities) = Visual, A = Auditory,
                          V (under Codes) = Verbal, S = Spatial
      Conflict Matrix: A and V in the first position = Auditory and Visual
                      V and S in the second position = Verbal and spatial
                      C = Cognitive, R = Response
Model limitations
------
1) assuming all tasks are alike
'''

import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib
# matplotlib.use("TkAgg")                         # for 3d cube visualization
# import matplotlib.backends.backend_tkagg as tkagg
from IPython import display
%matplotlib inline
import matplotlib.pyplot as plt

class MRT(tk.Frame):
	tasks = ['driving', 
			'reading-map', 
			'listening-navigation']
	tasksAnalysis = ['S PC+R V',
					'S PC+R V',
					'V PC A']
	taskAnalysisValues = [[1, 0, 1, 1, 0, 1], 
						[1, 0, 1, 1, 0, 1], 
						[0, 1, 1, 0, 1, 0]]     # taskAnalysisValues[task][Perception&Cognition Responding Visual Auditory Verbal Spatial]
	maxTaskSelection = 2
	indexSelectedTask = []		# index of 2 selected tasks

	def __init__(self, parent=None):
		tk.Frame.__init__(self, parent)
		self.pack()
		self.makeWidgets()
		self.taskAnalysis()
		self.cubeVis()
		self.interference()

	def makeWidgets(self):
		self.frameTaskSelection = tk.Frame(self)
		self.frameTaskAnalysis = tk.Frame(self)
		self.frameConflictMatrix = tk.Frame(self)
		self.frameCubeVis = tk.Frame(self)
		self.frameTaskSelection.grid(row=0, column=0, sticky=tk.NSEW)
		self.frameTaskAnalysis.grid(row=1, column=0, sticky=tk.NSEW)
		self.frameConflictMatrix.grid(row=2, column=0, sticky=tk.NSEW)
		self.frameCubeVis.grid(row=3, column=0, sticky=tk.NSEW)
		self.btnsTask = []
		for i in range(len(self.tasks)):
			n = tk.Button(self.frameTaskSelection, text=self.tasks[i], width=20)
			self.btnsTask.append(n)
			self.btnsTask[-1].grid(row=i, column=0, sticky=tk.NSEW)
			self.btnsTask[-1].bind('<ButtonPress-1>', self.taskSelection)
	
	def taskSelection(self, event):
		for i in range(len(self.tasks)):
			if event.widget is self.btnsTask[i]:
				if self.btnsTask[i].cget('bg') == 'green':
					self.btnsTask[i].config(bg='SystemButtonFace', fg='black')
					if i in self.indexSelectedTask:
						self.indexSelectedTask.remove(i)
				else:
					if len(self.indexSelectedTask) < self.maxTaskSelection:
						self.btnsTask[i].config(bg='green', fg='white')
						self.indexSelectedTask.append(i)
		for i in range(len(self.tasks)):
			if i in self.indexSelectedTask:
				self.tableTaskAnalysisShell.tag_configure(i, background='green', foreground='white')
			else:
				self.tableTaskAnalysisShell.tag_configure(i, background='white', foreground='black')
		self.cubeVis()
		self.interference()

	def taskAnalysis(self):
		self.tableTaskAnalysisShell = ttk.Treeview(self.frameTaskAnalysis, columns=('codes', 'stages', 'modalities'), selectmode="browse")
		self.labelTaskAnalysisTitle = tk.Label(self.frameTaskAnalysis, text="Task Analysis Shell", anchor=tk.CENTER)
		self.tableTaskAnalysisShell.grid(row=0, column=0)
		self.labelTaskAnalysisTitle.grid(row=1, column=0)
		self.tableTaskAnalysisShell.column('#0', width=100)
		self.tableTaskAnalysisShell.column('codes', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('stages', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('modalities', width=70, stretch=True)
		self.tableTaskAnalysisShell.heading('codes', text='Codes')
		self.tableTaskAnalysisShell.heading('stages', text='Stages')
		self.tableTaskAnalysisShell.heading('modalities', text='Modalities')
		for i in range(len(self.tasks)):
			self.tableTaskAnalysisShell.insert('', i, self.tasks[i], text=self.tasks[i], tags=i, values=self.tasksAnalysis[i])

	def cubeVis(self):
		pass

	def interference(self):
		pass


if __name__ == "__main__":
	root = tk.Tk()
	root.title("Multiple Resource Theory")
	MRT(root).mainloop()
	root.after(5, lambda: root.focus_force())

[1]
[1, 2]
[2]
[]
[1]
[1, 2]
[1, 2]
[1, 2]
[2]
[2, 0]
[0]
[0, 1]
